# 利用step1得到的user id，去爬user的facebook資訊

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import requests
from selenium.common.exceptions import NoSuchElementException
import itertools
from selenium import webdriver
import json

In [2]:
#使用登入facebook後的帳號進行爬蟲，得到的資訊會比較完整
service_args=[]
service_args.append('--load-images=no')
service_args.append('--disk-cache=yes')
service_args.append('--ignore-ssl-errors=true')
driver= webdriver.PhantomJS(r"C:\Users\X555LB\phantomjs-2.1.1-windows\bin\phantomjs.exe",service_args=service_args)
dic={}
try:
    driver.find_element_by_name('email').send_keys("")
    driver.find_element_by_name('pass').send_keys("")
    driver.find_element_by_name('login').click()
except:
    pass

In [14]:
#使用user id進行爬蟲
userlist=['10201882405008341',
 '816990981679144',
 '901596179856462',
 '1513373468885377',
 '859241384096525',
 '10152337619858258',
 '746844378694477',
 '10204913740710727',
 '10204569544541281',
 '803311276353254',
 '882108995132722',
 '487008184760969',
 '714818631888355',
 '795557530463344',
 '10152439960176588',
 '1504479799836368',
 '768281599871246',
 '798627530245583',
 '10152846769600842',
 '997704773577686',
 '833203480028553',
 '285440208511731']

c={}
c["users"]=[]

for user in userlist:
    dic={}
    driver= webdriver.PhantomJS(r"C:\Users\X555LB\phantomjs-2.1.1-windows\bin\phantomjs.exe",service_args=service_args)
    driver.get('https://www.facebook.com/'+user)
    start = time.time() 
    print(driver.current_url)
    
    # 試著登入
    try:
        driver.find_element_by_name('email').send_keys("")
        driver.find_element_by_name('pass').send_keys("")
        driver.find_element_by_name('login').click()
        driver.get_screenshot_as_file('test1.png')
    except:
        pass

    
    
    print(driver.current_url)
    

    time.sleep(1)
    try:
        driver.find_element_by_link_text('About').click()
    except:
        driver.refresh()
        time.sleep(2)
        driver.get_screenshot_as_file('test2.png')
        driver.find_element_by_link_text('About').click()
    if driver.current_url.split('.')[0].split('/')[-1]=="m":
        driver.get(driver.current_url.replace('/m.','/www.'))
        print(driver.current_url)

    conbs=bs(driver.page_source)

    aboutlist=['&section=education&pnref=about','&section=living&pnref=about','&section=contact-info&pnref=about','&section=relationship&pnref=about','&section=year-overviews&pnref=about']
    for about in aboutlist:
        driver.get(driver.current_url+about)

        conbs=bs(driver.page_source)

        # '''關於'''

        a=conbs.select('._4ms4')

        for content in a[0].select('._4qm1'):

            for title in content.select('._h72.lfloat._ohe._50f7'):
                dic[title.text]=[]

                if (title.text=="Work"):
                    for attr in content.select('._42ef'): #有資料的
                        for attrs in attr:
                            try:
                                dic[title.text].append({'work':attr.select('a')[0].text})
                            except:
                                pass    
                if (title.text == "Education") :
                    for attr in content.select('._42ef'): #有資料的
                        for attrs in attr:
                            try:
                                dic[title.text].append({"location":attr.select('.fsm.fwn.fcg')[0].text,'school_name':attr.select('a')[0].text})
                            except:
                                continue
                elif (title.text=="Current City and Hometown"):
                    for attr in content.select('._42ef'): #有資料的
                        for attrs in attr:
                            try:
                                dic[title.text].append({attr.select('.fsm.fwn.fcg')[0].text:attr.select('a')[0].text})
                            except:
                                pass
                elif (title.text=="Contact Information")or (title.text=="Basic Information"):
                    for attr in content.select('.clearfix._ikh'): #有資料的
                        try:
            
                            dic[title.text].append({attr.select('._50f4._5kx5')[0].text:attr.select('._4bl7._pt5')[0].text})
                        except:
                            continue

                elif (title.text=="Family Members"):
                    for attr in content.select('._42ef'): #有資料的
                        for attrs in attr:
                            try:
                                dic[title.text].append({attr.select('.fsm.fwn.fcg')[1].text:attr.select('.fsm.fwn.fcg')[0].text,'url':attr.select("a")[0]["href"]})
                            except:
                                continue
                else:
                    continue    


    driver.find_element_by_link_text("Photos").click()

    photo=bs(driver.page_source)
    photo_html=photo.select('._53s.fbPhotoCurationControlWrapper.fbPhotoStarGridElement.fbPhotoStarGridNonStarred.focus_target._53s.fbPhotoCurationControlWrapper')
    photo=[]
    for img in photo_html:
        photo.append({"url":img.select('i')[0]['style'].replace('(','*').replace(');','*').split('*')[1],'discribtion':img.select('a')[0]['aria-label']})
    dic["photo"]=photo                   

    driver.find_element_by_link_text('More').click()
    time.sleep(1)
    driver.get_screenshot_as_file('photo.png')
    more=bs(driver.page_source)
    url=(driver.current_url.split("sk=")[0])#先處理放for會跑版
    for mores in more.select('._54nh') :
        more_title=mores.text

        if more_title== "Check-Ins":
            more_title="map"
        dic[more_title]=[]
        time.sleep(1)
        driver.get((url+"sk="+str.lower(more_title)).split(" ")[0])
        conbs=bs(driver.page_source)
        try:
            driver.get_screenshot_as_file(more_title+'.png')
            if conbs.select("._4-y-")[0].text=="No activity to show":
                continue
        except:
            try:
                conbs=bs(driver.page_source)

                if str.lower(more_title) =="likes":
                    content=(conbs.select("._5h60._30f"))
                    for contents in content[0].select("._42ef"):
                        dic[more_title].append({"name":contents.select('.fsl.fwb.fcb')[0].text,'url':contents.select('a')[0]['href'],'type':contents.select('.fsm.fwn.fcg')[0].text,'id':contents.select('a')[0]['data-hovercard'].split('?id=')[1]})
                elif str.lower(more_title)=="events":

                    content=(conbs.select(".uiList._153e._509-._4ki"))

                    for contents in content[0].select("._4cbv"):
                        dic[more_title].append({"name":contents.select('a')[0].text,'url':'https://www.facebook.com'+contents.select('a')[0]['href'],'time':contents.select('._4cbu.fsm.fwn.fcg')[0].text})
                elif str.lower(more_title)=="map":
                    content=(conbs.select(".uiList._620._14b9"))
                    for contents in content[0].select("._5rz"):
                        dic[more_title].append({"name":contents.select('._gx7')[0].text,'url':contents.select('._gx7')[0]['href']})
                elif str.lower(more_title)=="reviews":
                    content=(conbs.select("._5h60._30f"))
                    for contents in content[0].select("._5mt_"):
                        dic[more_title].append({"url":contents.select('a')[0]["href"],"name":contents.select('a')[0].text,"stars":contents.select('i')[0].text,"time":contents.select('abbr')[0]["title"]})
                elif str.lower(more_title)=="groups":
                    conbs=bs(driver.page_source)
                    content=(conbs.select("._5h60._30f"))
                    for contents in content[0].select(".mbs.fwb"):
                        dic[more_title].append({'url':"https://www.facebook.com"+contents.select('a')[0]["href"],'name':contents.select('a')[0].text})


                else:
                    content=(conbs.select(".uiList._620._14b9"))
                    for contents in content[0].select("._5rz"):
                        dic[more_title].append({'id':contents.select('._3owb')[0]['data-obj'],'url':contents.select('a')[0]['href'],'name':contents.select('._gx7')[0].text,'type':contents.select('._1fs8.fsm.fwn.fcg')[0].text})
            except:
                continue


    end = time.time()
    elapsed = end - start
    print("時間: "+str(elapsed))
    driver.close()
    for x in list(dic.keys()):
            if dic[x] == [] :
                del dic[x]
    c["users"].append(dic)


https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2F10201882405008341
https://www.facebook.com/10201882405008341
https://www.facebook.com/blues.f.hu/about?lst=100002281396634%3A1429993351%3A1501343898


C:\Users\X555LB\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\X555LB\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


1
1
時間: 57.22272515296936
https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2F816990981679144
https://www.facebook.com/profile.php?id=100001047488031
1
1
時間: 52.34106230735779
https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2F901596179856462
https://www.facebook.com/profile.php?id=100000182661336
1
1
時間: 54.83715558052063
https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2F1513373468885377
https://m.facebook.com/mike.ma.5203
https://www.facebook.com/mike.ma.5203/about?lst=100002281396634%3A100006382142630%3A1501344079
1
1
時間: 60.57251739501953
https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2F859241384096525
https://m.facebook.com/josephhuang007
https://www.facebook.com/josephhuang007/about?lst=100002281396634%3A100000320155903%3A1501344146
1
1
時間: 67.61097574234009
https://www.facebook.com/login.php?next=https%3A%2F%2Fwww.facebook.com%2F10152337619858258
https://m.facebook.com/davidwangtaipei
https

In [15]:
c['users']

[{'Basic Information': [{'Gender': 'Male'},
   {'Languages': '台語 · 中文 · 英文'},
   {'Religious Views': '無神論者'},
   {'Political Views': '灰色地帶灰色地帶'}],
  'Current City and Hometown': [{'Current city': 'Darwin, Northern Territory'},
   {'Hometown': 'Taipei, Taiwan'}],
  'photo': [{'discribtion': 'No automatic alt text available.',
    'url': 'https://scontent-tpe1-1.xx.fbcdn.net/v/t1.0-0/q87/p206x206/19059458_10209556523056496_8758838668611546535_n.jpg?oh=44344142218a25ca53d24622c8fef116&oe=59FA613E'},
   {'discribtion': 'Image may contain: sky, tree, basketball court and outdoor',
    'url': 'https://scontent-tpe1-1.xx.fbcdn.net/v/t1.0-0/q82/p206x206/18893375_10209490976377870_4225608922057665441_n.jpg?oh=e3541b301fbf7773ed57ad29a3e55fd0&oe=5A34F5C8'},
   {'discribtion': 'Image may contain: indoor',
    'url': 'https://scontent-tpe1-1.xx.fbcdn.net/v/t1.0-0/p206x206/18620376_10209400927486704_1779000808656452805_n.jpg?oh=1b4a3ab70392f32ad7cfec8bc2e92851&oe=5A01013D'},
   {'discribtion': 'Ima

In [16]:
len(c['users'])

22

In [17]:
json_c=json.dumps(c,ensure_ascii=False,indent=2)
print(json_c)

{
  "users": [
    {
      "Current City and Hometown": [
        {
          "Current city": "Darwin, Northern Territory"
        },
        {
          "Hometown": "Taipei, Taiwan"
        }
      ],
      "Basic Information": [
        {
          "Gender": "Male"
        },
        {
          "Languages": "台語 · 中文 · 英文"
        },
        {
          "Religious Views": "無神論者"
        },
        {
          "Political Views": "灰色地帶灰色地帶"
        }
      ],
      "photo": [
        {
          "url": "https://scontent-tpe1-1.xx.fbcdn.net/v/t1.0-0/q87/p206x206/19059458_10209556523056496_8758838668611546535_n.jpg?oh=44344142218a25ca53d24622c8fef116&oe=59FA613E",
          "discribtion": "No automatic alt text available."
        },
        {
          "url": "https://scontent-tpe1-1.xx.fbcdn.net/v/t1.0-0/q82/p206x206/18893375_10209490976377870_4225608922057665441_n.jpg?oh=e3541b301fbf7773ed57ad29a3e55fd0&oe=5A34F5C8",
          "discribtion": "Image may contain: sky, tree, basketball c

In [9]:
driver.get('https://www.facebook.com/859241384096525')

try:
    driver.find_element_by_name('email').send_keys("kevin19961009@gmail.com")
    driver.find_element_by_name('pass').send_keys("kkkkk5553705")
    driver.find_element_by_name('login').click()

except:
    pass
# driver.get('https://www.facebook.com/josephhuang007')

print(driver.current_url)
driver.get_screenshot_as_file('test1.png')

time.sleep(1)

driver.find_element_by_link_text('About').click()
print(driver.current_url)
    


# print(bs(driver.page_source))

https://m.facebook.com/josephhuang007
https://m.facebook.com/josephhuang007/about?lst=100002281396634%3A100000320155903%3A1501172575


In [16]:
a=['10201882405008341',
 '816990981679144',
 '901596179856462',
 '1513373468885377',
 '859241384096525',
 '10152337619858258',
 '746844378694477',
 '10204913740710727',
 '10204569544541281',
 '803311276353254',
 '882108995132722',
 '487008184760969',
 '714818631888355',
 '795557530463344',
 '10152439960176588',
 '1504479799836368',
 '768281599871246',
 '798627530245583',
 '10152846769600842',
 '997704773577686',
 '833203480028553',
 '285440208511731']
# from PIL import Image                                                                                

for aa in a:
    driver.get('http://www.facebook.com/'+aa)
#     print(driver.current_url)
    time.sleep(1)
    print(driver.current_url)
    try:
        driver.find_element_by_link_text('About').click()
    except:
        time.sleep(5)
        driver.get_screenshot_as_file('test1.png')
        print('bad')
        driver.find_element_by_link_text('About').click()

    if driver.current_url.split('.')[0].split('/')[-1]=="m":
#         print('m')
        driver.get(driver.current_url.replace('/m.','/www.'))
#         print(driver.current_url.replace('/m.','/www.'))
    print(driver.current_url)

https://www.facebook.com/blues.f.hu
bad
https://www.facebook.com/blues.f.hu/about?lst=100002281396634%3A1429993351%3A1501316524
https://www.facebook.com/profile.php?id=100001047488031
https://www.facebook.com/profile.php?id=100001047488031&lst=100002281396634%3A100001047488031%3A1501316532&sk=about
https://www.facebook.com/profile.php?id=100000182661336
https://www.facebook.com/profile.php?id=100000182661336&lst=100002281396634%3A100000182661336%3A1501316536&sk=about
https://m.facebook.com/mike.ma.5203
bad
https://www.facebook.com/mike.ma.5203/about?lst=100002281396634%3A100006382142630%3A1501316539
https://m.facebook.com/josephhuang007
https://www.facebook.com/josephhuang007/about?lst=100002281396634%3A100000320155903%3A1501316548
https://m.facebook.com/davidwangtaipei
https://www.facebook.com/davidwangtaipei/about?lst=100002281396634%3A563388257%3A1501316554
https://m.facebook.com/mike.seven.007
https://www.facebook.com/mike.seven.007/about?lst=100002281396634%3A100001067657929%3A150